# 4.2.2 Heuristic Filtering

In [ ]:
import json
import re
word_freq_file = "./projects/m2conceptbase/utils/word_pos_freq_train_data_230514.txt"


def get_word_freq_dict(filename="./projects/m2conceptbase/word_freq_test_data_230102.txt"):
    word_freq_dict = {}
    with open(filename, mode='r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            items = line.strip().split('\t')
            # print(items)
            if len(items) != 3:
                continue
            if items[0].strip() != items[0]:
                continue
            if items[2] not in word_freq_dict:
                word_freq_dict[items[2]] = {}
                word_freq_dict[items[2]][items[0]] = int(items[1])
            else:
                word_freq_dict[items[2]][items[0]] = int(items[1])
    
    return word_freq_dict

word_freq_dict = get_word_freq_dict(word_freq_file)
# word_freq_dict

for key, value in word_freq_dict.items():
    l = sorted(value.items(), key=lambda x:x[1], reverse=True)
    word_freq_dict[key] = {word: count for word, count in l}

with open("word_freq_train_data_230514_v2.json", 'w', encoding='utf-8') as f:
    json.dump(word_freq_dict, f, ensure_ascii=False, indent=4)

print("saved.")

saved.


In [ ]:
with open("word_freq_train_data_230514_v2.json", 'r', encoding='utf-8') as f:
    word_freq_dict = json.load(f)

stopwords = {}.fromkeys([line.rstrip() for line in open('./data_122/kg_data/stopwords.txt', 'r', encoding='utf-8')])
excluded_char = ['\\', '/', '?', '', '.', ';', '=', '*', '^', '>', '<', '#', '$', '@', ')', '(', '&', '丨', '%', '`', '\t', ' ']
for i in stopwords:
    pattern = re.compile(r'[^\u4e00-\u9fa5a-zA-Z]')
    if not re.sub(pattern, '', i):
        excluded_char.append(i)
excluded_char = list(set(excluded_char)) 
excluded_concepts = ["图片", "大图", "照片", "版", "们", "子", "时", "人名", "暨", "令人"]

# 返回True表示要删除
def check_excluded_char(concept):
    if len(concept) == 1 and (65 <= ord(concept) <= 90 or 97 <= ord(concept) <= 122):
        return True
    if concept in excluded_concepts: # 人工排除的概念
        return True
    if any(char in excluded_char for char in concept):   # 包含停用词
        # print(concepts)
        return True
    else:
        return False

cnt = 0
word_freq_dict_new = {}
for pos, word_freq in word_freq_dict.items():
    word_freq_dict_new[pos] = word_freq.copy()
    for concept, count in word_freq.items():
        if check_excluded_char(concept):
            cnt += 1
            # print(concept)
            del(word_freq_dict_new[pos][concept])

print(cnt)

with open("word_freq_train_data_230514_v3.json", 'w', encoding='utf-8') as f:
    json.dump(word_freq_dict_new, f, ensure_ascii=False, indent=4)
print("saved.")

20383
saved.


In [36]:
import json
with open("word_freq_train_data_230514_v3.json", 'r', encoding='utf-8') as f:
    word_freq_dict = json.load(f)
# word_freq_dict

def is_en(s):
    return s.encode('utf-8').isalpha()


word_freq_dict_new = {}
for pos, word_freq in word_freq_dict.items():

    if pos == 'ng':
        continue
    
    if pos == 'nrfg' or pos == 'nrt' or pos == 'nr':
        temp_list = list(word_freq_dict[pos].items())
        temp_list = [item for item in temp_list if len(item[0]) > 1] # 去除单字
        word_freq_dict_new[pos] = dict(temp_list[:50])
    
    if pos == 'nw': # top 300
        temp_list = list(word_freq_dict[pos].items())
        temp_list = [item for item in temp_list if len(item[0]) > 1] # 去除单字
        word_freq_dict_new[pos] = dict(temp_list[:300])
        
    if pos == 'nt':
        temp_list = list(word_freq_dict[pos].items())
        temp_list = [item for item in temp_list if len(item[0]) > 1] # 去除单字
        temp_list = [item for item in temp_list if item[1] >= 400]
        word_freq_dict_new[pos] = dict(temp_list)
    
    if pos == 'ns':
        temp_list = list(word_freq_dict[pos].items())
        temp_list = [item for item in temp_list if len(item[0]) > 1] # 去除单字
        temp_list = [item for item in temp_list if item[1] >= 3000]
        word_freq_dict_new[pos] = dict(temp_list)
    
    if pos == 'nz':
        temp_list = list(word_freq_dict[pos].items())
        temp_list = [item for item in temp_list if len(item[0]) > 1] # 去除单字
        temp_list_en = [item for item in temp_list if is_en(item[0])][:50]
        temp_list_zh = [item for item in temp_list if not is_en(item[0])]
        word_freq_dict_new[pos] = dict(temp_list_zh + temp_list_en)
    
    if pos == 'n':
        temp_list = list(word_freq_dict[pos].items())
        temp_list = [item for item in temp_list if not is_en(item[0])]
        word_freq_dict_new[pos] = dict(temp_list)
    
with open("word_freq_train_data_230514_v4.json", 'w', encoding='utf-8') as f:
    json.dump(word_freq_dict_new, f, ensure_ascii=False, indent=4)
print("saved.")

saved.


In [41]:
import json, re
with open("word_freq_train_data_230514_v4.json", 'r', encoding='utf-8') as f:
    word_freq_dict = json.load(f)
    
def is_en_zh_mixed(concept):
    has_en = False
    has_zh = False
    
    for char in concept:
        if '\u4e00' <= char <= '\u9fa5':
            has_zh = True
        elif 'a' <= char <= 'z' or 'A' <= char <= 'Z':
            has_en = True
            
        if has_zh and has_en:
            return True
    return False

word_freq_dict_new = {}
keep_list = [
    "t恤",
    "v领",
    "b超",
    "k线",
    "u盘",
    "a股",
    "polo衫",
    "bb霜",
    "c语言",
    "cp感",
    "pos机",
    "up主",
    "a股市场",
    "led灯",
    "kt板",
    "t台",
    "k歌",
    "pc端",
    "ic卡",
    "x射线",
    "卡拉ok",
    "维生素a",
    "b超单",
    "c罗",
    "atm机",
    "led屏",
    "维生素c",
    "芒果tv",
    "hpv疫苗",
    "ph值",
    "v脸",
    "真人cs",
    "维生素d",
    "哆啦a梦",
    "x战警"
]

cnt = 0
word_freq_dict_new = {}

for pos, word_freq in word_freq_dict.items():
    word_freq_dict_new[pos] = word_freq.copy()
    for word, freq in word_freq.items():
        if is_en_zh_mixed(word):
            if word not in keep_list:
                # print(word, freq)
                cnt += 1
                del(word_freq_dict_new[pos][word])
            # if len([1 for end in ["级", "型", "类", "形", "卷", "版", "柱", "皮", "男", "l", "资"] if word.endswith(end)]) > 0:
            #     # print(word, freq)
            #     pass
            # elif freq > 1000: 
            #     print("\""+word+"\""+",")
print(cnt)

    
with open("word_freq_train_data_230514_v5.json", 'w', encoding='utf-8') as f:
    json.dump(word_freq_dict_new, f, ensure_ascii=False, indent=4)
print("saved.")

7531
saved.


In [2]:
import json
with open("word_freq_train_data_230514_v6.json", 'r', encoding='utf-8') as f:
    word_freq_dict = json.load(f)
    
word_freq_dict_new = {}

for pos, word_freq in word_freq_dict.items():
    word_freq_dict_new[pos] = word_freq.copy()
    for word, freq in word_freq.items():
        if freq < 20:
            # print(word, freq)
            del(word_freq_dict_new[pos][word])

with open("word_freq_train_data_230514_v7.json", 'w', encoding='utf-8') as f:
    json.dump(word_freq_dict_new, f, ensure_ascii=False, indent=4)
print("saved.")

saved.


`word_freq_train_data_230514_v7.json` is a word_freq file with larger frequency (20) threshod.

In [27]:
def check(s):
    import re
    my_re = re.compile(r'[A-Za-z]',re.S)
    res = re.findall(my_re, s)
    if len(res):
        return True
    return False
# check("dasc偶奇偶")
s = "aAsds".encode('utf-8')
s.isalpha()


True

In [39]:
def test(concept):
    if len(concept) == 1 and (65 <= ord(concept) <= 90 or 97 <= ord(concept) <= 122):
        return True
    return False
test("ab")

False

we use `word_freq_train_data_230514_v6.json` to get the final word freq file.

In [4]:
import json
with open("word_freq_train_data_230514_v6.json", 'r', encoding='utf-8') as f:
    word_freq_dict = json.load(f)
    
word_freq_dict_new = {}

for pos, word_freq in word_freq_dict.items():
    for word, freq in word_freq.items():
        word_freq_dict_new[word] = word_freq_dict_new.get(word, 0) + freq


li = sorted(word_freq_dict_new.items(), key=lambda x:x[1], reverse=True)
word_freq_dict_new = {word: count for word, count in li}

with open("word_freq_train_data_230531.json", 'w', encoding='utf-8') as f:
    json.dump(word_freq_dict_new, f, ensure_ascii=False, indent=4)
print("saved.")

saved.
